In [3]:
# The values for K are is derived from the formula:  abs(sin(i+ 1))×(2**32)
K = [
0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821,
0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa,
0xd62f105d, 0x02441453, 0xd8a1e681, 0xe7d3fbc8,
0x21e1cde6, 0xc33707d6, 0xf4d50d87, 0x455a14ed,
0xa9e3e905, 0xfcefa3f8, 0x676f02d9, 0x8d2a4c8a,
0xfffa3942, 0x8771f681, 0x6d9d6122, 0xfde5380c,
0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70,
0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x04881d05,
0xd9d4d039, 0xe6db99e5, 0x1fa27cf8, 0xc4ac5665,
0xf4292244, 0x432aff97, 0xab9423a7, 0xfc93a039,
0x655b59c3, 0x8f0ccc92, 0xffeff47d, 0x85845dd1,
0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1,
0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391
]

In [4]:
# number of shifts
shift = [
    7, 12, 17, 22,
    7, 12, 17, 22,
    7, 12, 17, 22,
    7, 12, 17, 22,
    5, 9, 14, 20,
    5, 9, 14, 20,
    5, 9, 14, 20,
    5, 9, 14, 20,
    4, 11, 16, 23,
    4, 11, 16, 23,
    4, 11, 16, 23,
    4, 11, 16, 23,
    6, 10, 15, 21,
    6, 10, 15, 21,
    6, 10, 15, 21,
    6, 10, 15, 21
]

The MD5  F, G, H and I functions

In [5]:
def F(B, C, D):
    return (B & C) | ((~B) & D)


In [6]:
def G(B,C,D):
    return (B & D)| (C & (~D))

In [7]:
def H(B, C, D):
    return B ^ C ^ D

In [8]:
def I(B, C, D):
    return C ^ (B | (~D))

Modular addition

In [9]:
def mod_add(X,Y):
    Z=0x100000000
    return (X+Y)%Z

Left rotate

In [10]:
def left_rotate(x, amount):
    x &= 0xFFFFFFFF
    return ((x<<amount) | (x>>(32-amount))) & 0xFFFFFFFF

Round 
###### m_index=[
######  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
######    [1,6,11,0,5,10,15,4,9,14,3,8,13,2,7,12],
######    [5,8,11,14,1,4,7,10,13,0,3,6,9,12,15,2],
######    [0,7,14,5,12,3,10,1,8,15,6,13,4,11,2,9]
###### ]
    

In [11]:
def round(A, B, C, D,M, round_num):
    for i in range(16):
        if round_num == 1:
            fun_val = F(B, C, D)
            m_index = i
        elif round_num == 2:
            fun_val = G(B, C, D)
            m_index = (5*i + 1) % 16
        elif round_num == 3:
            fun_val = H(B, C, D)
            m_index = (3*i + 2) % 16
        else: 
            fun_val = I(B, C, D)
            m_index = (7*i) % 16

        fun_val = mod_add(fun_val, A)
        fun_val = mod_add(fun_val, K[i + 16*(round_num-1)])
        fun_val = mod_add(fun_val, int.from_bytes(M[m_index*4:(m_index+1)*4], byteorder='little'))  #???

        new_B = mod_add(B , left_rotate(fun_val, shift[i + 16*(round_num-1)])) 
        A, B, C, D = D, new_B, B, C

    return A, B, C, D

implement MD5 algorithm

In [12]:
def MD5(message):
    # Convert the string to binary
    bin_string = ''.join(format(ord(c), '08b') for c in message)
    
    # Add padding
    bin_string += '1'
    len_bits = len(bin_string)
    padding_needed = (448 - len_bits) % 512
    bin_string += '0' * padding_needed
    
    # Add length
    original_length = format(len(message) * 8, '064b')
    bin_string += original_length

    padded_message = bin_string

    # Convert the binary string to bytes
    message_bytes = int(padded_message, 2).to_bytes(len(padded_message) // 8, byteorder='little')

    # initialization vectors 
    A =0x01234567
    B =0x89abcdef
    C =0xfedcba98
    D =0x76543210
  

    # Process each 512-bit chunk
    for i in range(0, len(message_bytes), 64):
        M = message_bytes[i:i+64]
        A_temp, B_temp, C_temp, D_temp = A, B, C, D
        for round_num in range(1, 5):
            A_temp, B_temp, C_temp, D_temp = round(A_temp, B_temp, C_temp, D_temp, M, round_num)
            A = mod_add(A,A_temp)
            B = mod_add(B,B_temp)
            C = mod_add(C,C_temp)
            D = mod_add(D,D_temp)
           
    
    return (A.to_bytes(4, byteorder='little') +
            B.to_bytes(4, byteorder='little') +
            C.to_bytes(4, byteorder='little') +
            D.to_bytes(4, byteorder='little')).hex()


Round1 

In [13]:
print(MD5('Hello world'))
    

a8d05dc0de4f81236fbd2bed92529353


In [27]:
from collections import deque
class Node:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.children = set()
        self.siblings = set()
        self.connections = set()

class FamilyTree:
    def __init__(self):
        self.nodes = {}                
        self.node_indices = {}
        self.index_counter = 0

    def add_node(self, child_name, parent_name=None):
        if parent_name is not None and parent_name not in self.nodes:
            self.nodes[parent_name] = Node(parent_name)
            self.node_indices[parent_name] = self.index_counter
            self.index_counter += 1
        if child_name not in self.nodes:
            self.nodes[child_name] = Node(child_name, parent_name)
            self.node_indices[child_name] = self.index_counter
            self.index_counter += 1

        if parent_name is not None:
            parent_node = self.nodes[parent_name]
            child_node = self.nodes[child_name]
            parent_node.children.add(child_node)
            child_node.parent = parent_node
            parent_node.connections.add(child_node)
            child_node.connections.add(parent_node)

            for sibling in parent_node.children:
                if sibling != child_node:
                    child_node.siblings.add(sibling)
                    sibling.siblings.add(child_node)


    def find(self, name):
        return name in self.nodes

    def delete_node(self, name):
        if not self.find(name):
            raise ValueError(f"Node '{name}' does not exist.")
        else:
            if len(self.nodes[name].children) != 0:
                for child in self.nodes[name].children:
                    self.nodes[child.name].parent = None
            del self.nodes[name]

    def sibling(self, name1, name2):
        if self.find(name1) and self.find(name2):
            return name1 in self.nodes[name2].siblings
        else:
            raise ValueError("One or both nodes do not exist.")

    def is_ancestor(self, ancestor, descendant):
        if self.find(ancestor) and self.find(descendant):
            while descendant is not None:
                if descendant in self.nodes[ancestor].children:
                    return True
                descendant = self.nodes[descendant].parent
            return False
        else:
            raise ValueError("One or both nodes do not exist.")

    def common_Parent(self, name1, name2):
        if self.find(name1) and self.find(name2):
            ancestors1 = set()
            current = name1
            while current is not None:
                ancestors1.add(current)
                current = self.nodes[current].parent

            current = name2
            while current is not None:
                if current in ancestors1:
                    return current
                current = self.nodes[current].parent
        else:
            raise ValueError("One or both nodes do not exist.")

    def has_relation(self,name1,name2):
        if self.find(name1) and self.find(name2):
            if self.common_Parent(name1,name2) and not (self.is_ancestor(name1,name2) and self.is_ancestor(name2,name1)):
                return True 
            return False
        else:
            raise ValueError("One or both nodes do not exist.")

    def furthest_child(self, name):
        if name in self.nodes:
            node = self.nodes[name]

            if node.children:
                max_depth, furthest_child = max((self.furthest_child(child.name) for child in node.children), key=lambda x: x[0])
                return 1 + max_depth, furthest_child
            else:
                return 0, node
        else:
            raise ValueError("One or both nodes do not exist.")

    def __find_root(self):
        for name, node in self.nodes.items():
            if node.parent is None:
                return name
        return None
     

    def bfs(self, start):
        visited = {name: False for name in self.nodes}
        distance = {name: 0 for name in self.nodes}
        queue = deque([start])
        visited[start] = True

        while queue:
            node = queue.popleft()
            for neighbour in self.nodes[node].connections:
                if not visited[neighbour.name]:
                    queue.append(neighbour.name)
                    visited[neighbour.name] = True
                    distance[neighbour.name] = distance[node] + 1

        max_distance_node = max(distance, key=distance.get)
        return max_distance_node, distance[max_distance_node]

    def find_diameter(self):
        root = self.__find_root()
        # First BFS to find one end point of the longest path
        node, _ = self.bfs(root)

        # Second BFS to find the actual longest path
        second_node, diameter = self.bfs(node)

        return node,second_node,diameter




In [28]:
tree=FamilyTree()
tree.add_node('A','B')
tree.add_node('B','C')
tree.add_node('D','C')
tree.add_node('E','D')

In [25]:
import matplotlib.pyplot as plt
import networkx as nx

def draw_tree():
 
    graph_data = {}
    
    for name, node in tree.nodes.items():
        childs = []
        for child in node.children:
            childs.append(child.name)
        
        graph_data[name] = childs
        
    G = nx.from_dict_of_lists(graph_data, create_using=nx.DiGraph)

    pos = nx.spring_layout(G)  
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='pink', font_size=10, font_color='black', font_weight='bold', edge_color='gray', linewidths=1, arrowsize=20)

    plt.show()

draw_tree()  



In [18]:
dist,node=tree.furthest_child('C')
print(f'distance: {dist}')
print(f'furthest child: {node.name}')



distance: 2
furthest child: E


In [31]:
first,second,dist=tree.find_diameter()

print(f'the furthest relation is between {first} and {second}  distance={dist}')


the furthest relation is between A and E  distance=4
